In [ ]:
## Python course for data scientists in a moderate level, supported by Wagatsuma Lab@Kyutech 
#
# The MIT License (MIT): Copyright (c) 2022 Hiroaki Wagatsuma and Wagatsuma Lab@Kyutech
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
# The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE. */
#
# # @Time    : 2022-03-27 
# # @Author  : Hiroaki Wagatsuma
# # @Site    : https://github.com/hirowgit/2B3_python_owl_logic_database_course
# # @IDE     : Python 3.9.7 (default, Oct 17 2021, 16:26:06) [Clang 10.0.0 (clang-1000.11.45.5)] on darwin
# # @File    : lec1_step5.py 

In [70]:
import requests
import xml.etree.ElementTree as xmlet
import re
import numpy as np
import sys
import os

fname_tag="mpd02"
fext=".txt"

#pubmed URL
url_header="https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
url_footer=["esearch.fcgi","efetch.fcgi"]

#backup data
fout_name="fout_mpd_fullLen02.txt"

fkeys=["marmoset","parkinson","disease"]
conj=" AND "
kterm=fkeys[0]
for label in fkeys[1:len(fkeys)]:
    kterm=kterm+conj+label
    print(kterm)

# search
pubmed_search_url = url_header+url_footer[0]
query_keys = {
    "db":"pubmed",
#     "term":'1-methyl-4-phenyl-1,2,3,6-tetrahydropyridine (MPTP)-lesioned common marmoset used',
    "term":'common marmoset',
    "usehistory":"y"
    # keeping data for the next fetch action
}
print(query_keys)
resDf = requests.get(pubmed_search_url, params=query_keys)
xmlD = resDf.text.encode("utf-8")
targetD = xmlet.fromstring(xmlD)
# print(resDf)
# print(xmlD)

# dig and get from target ID obtained the previous action
pubmed_fetch_url = url_header+url_footer[1]
query_keys = {
    "db" : "pubmed",
    "query_key" : targetD.findtext("QueryKey"),
    "WebEnv" : targetD.findtext("WebEnv"),
    "retmode" : "text",
    "rettype" : "abstract"
}

resDg = requests.get(pubmed_fetch_url, params=query_keys)
with open(fout_name, "w", encoding="utf-8") as fid:
    fid.write(resDg.text)
    
resDg = requests.get(pubmed_fetch_url, params=query_keys)
strAnalysis=resDg.text

pat_header ='<eSearchResult><Count>'
pat_body ='.*(\d+)'
pat_footer ='</Count>'
patternRe =pat_header+pat_body+pat_footer

resOri = re.search(patternRe, resDf.text)
resNumStr=resOri.group()
resNum=int(resNumStr[len(pat_header):-len(pat_footer)])
print(resNum)

blockPos=[]
patternRep="\n(\d+). "
res1st =re.search(patternRep, strAnalysis)
# print(res1st.group())
# print(res1st)
Mstart, Mend = res1st.span()
blockPos.append([Mstart,Mend])
# print([Mstart,Mend])
    
patternRep="\n\n(\d+). "
res_iter =re.finditer(patternRep, strAnalysis)
for res in res_iter:
#     print(res.group())
#     print(res)
    Mstart, Mend = res.span()
    blockPos.append([Mstart,Mend])
#     print([Mstart,Mend])

# len(blockPos)

npblockPos=np.array(blockPos)
blockST=npblockPos[:,0].tolist()
npblockPos[-1,-1]

npblockED=npblockPos[1:npblockPos.shape[0],0]+1
blockED=npblockED.tolist()
blockED.append(len(strAnalysis))

# print(blockED)

if not os.path.isdir(fname_tag):
        os.makedirs(fname_tag)

i=0
k=0
while k<resNum:
    fout_name=os.path.join(fname_tag,str(k+1)+fext)
    patternRep="\n\n(\d+). "
    if i==0:
        patternRep="\n(\d+). "
    else:
        patternRep="\n\n(\d+). "  

    strAnalysis=resDg.text[blockST[i]:blockED[i]]
    res =re.search(patternRep, strAnalysis)
    if i==0:
        hpos=1
    else:
        hpos=2 
    numKey0=int(res.group()[hpos:-2])
#     print(numKey0)
    pre_i=i
    i=i+1
    patternRep="\n\n(\d+). "
    strAnalysis=resDg.text[blockST[i]:blockED[i]]
    res =re.search(patternRep, strAnalysis)
    numKey1=int(res.group()[2:-2])

    while (k+2)!=numKey1 and k!=resNum-1:
        i=i+1
        strAnalysis=resDg.text[blockST[i]:blockED[i]]
        res =re.search(patternRep, strAnalysis)
        numKey1=int(res.group()[2:-2])
    if i>=len(blockED)-1:
        i=len(blockED)
        
    strAnalysis=resDg.text[blockST[pre_i]:blockED[i-1]]
    fout_name=os.path.join(fname_tag,str(k+1)+fext)
    with open(fout_name, "w", encoding="utf-8") as fid:
        fid.write(strAnalysis)
    
    k=k+1
    print([k, i])
    print(strAnalysis[1:10])
    print("---")


marmoset AND parkinson
marmoset AND parkinson AND disease
{'db': 'pubmed', 'term': 'common marmoset', 'usehistory': 'y'}
5023
[1, 1]
1. J Med 
---
[2, 2]

2. Sci R
---
[3, 3]

3. Emerg
---
[4, 4]

4. Sci R
---
[5, 5]

5. J Med
---
[6, 6]

6. J Med
---
[7, 7]

7. Behav
---
[8, 8]

8. Lab A
---
[9, 9]

9. Neuro
---
[10, 10]

10. Fron
---
[11, 11]

11. Cell
---
[12, 12]

12. Mol 
---
[13, 13]

13. Neur
---
[14, 14]

14. Comm
---
[15, 15]

15. Gero
---
[16, 16]

16. NPJ 
---
[17, 17]

17. Sci 
---
[18, 18]

18. Fron
---
[19, 19]

19. Sci 
---
[20, 20]

20. Elif
---
[21, 21]

21. Infe
---
[22, 22]

22. J Co
---
[23, 23]

23. Tran
---
[24, 24]

24. J Am
---
[25, 25]

25. Acta
---
[26, 26]

26. J Ex
---
[27, 27]

27. Bioc
---
[28, 28]

28. ILAR
---
[29, 29]

29. J Vi
---
[30, 30]

30. PLoS
---
[31, 31]

31. Agin
---
[32, 32]

32. Nat 
---
[33, 33]

33. J Co
---
[34, 34]

34. Vet 
---
[35, 35]

35. Biom
---
[36, 36]

36. Ecoh
---
[37, 37]

37. Vacc
---
[38, 38]

38. Micr
---
[39, 39]

39. Nat 

[333, 333]

333. J N
---
[334, 334]

334. Gen
---
[335, 335]

335. BMC
---
[336, 336]

336. J H
---
[337, 337]

337. Mol
---
[338, 338]

338. Pat
---
[339, 339]

339. Fro
---
[340, 340]

340. mSy
---
[341, 341]

341. Ana
---
[342, 342]

342. Sci
---
[343, 343]

343. Ste
---
[344, 344]

344. Rev
---
[345, 345]

345. Eur
---
[346, 346]

346. Ste
---
[347, 347]

347. Bio
---
[348, 348]

348. J N
---
[349, 349]

349. Sci
---
[350, 350]

350. Bio
---
[351, 351]

351. Nat
---
[352, 352]

352. Pri
---
[353, 353]

353. Pri
---
[354, 354]

354. Cer
---
[355, 355]

355. Ani
---
[356, 356]

356. BMC
---
[357, 357]

357. Hep
---
[358, 358]

358. Sci
---
[359, 359]

359. Par
---
[360, 360]

360. Exp
---
[361, 361]

361. PLo
---
[362, 362]

362. J P
---
[363, 363]

363. iSc
---
[364, 364]

364. Neu
---
[365, 365]

365. PLo
---
[366, 366]

366. Bio
---
[367, 367]

367. Mob
---
[368, 368]

368. Am 
---
[369, 369]

369. J M
---
[370, 370]

370. Bio
---
[371, 371]

371. Eur
---
[372, 372]

372. Dru
---


[662, 663]

662. Phy
---
[663, 664]

663. J A
---
[664, 665]

664. Nat
---
[665, 666]

665. J R
---
[666, 667]

666. Neu
---
[667, 668]

667. Int
---
[668, 669]

668. Eur
---
[669, 670]

669. Com
---
[670, 671]

670. IEE
---
[671, 672]

671. J V
---
[672, 673]

672. Sci
---
[673, 674]

673. J C
---
[674, 675]

674. ACS
---
[675, 676]

675. Neu
---
[676, 677]

676. Nat
---
[677, 678]

677. Ani
---
[678, 679]

678. PLo
---
[679, 680]

679. PLo
---
[680, 681]

680. Mic
---
[681, 682]

681. Nat
---
[682, 683]

682. Ant
---
[683, 684]

683. Ana
---
[684, 685]

684. Am 
---
[685, 686]

685. J P
---
[686, 687]

686. Mol
---
[687, 688]

687. Cel
---
[688, 689]

688. Hea
---
[689, 690]

689. Nat
---
[690, 691]

690. PLo
---
[691, 692]

691. eNe
---
[692, 693]

692. Phy
---
[693, 694]

693. Sci
---
[694, 695]

694. Par
---
[695, 696]

695. Psy
---
[696, 697]

696. Bra
---
[697, 698]

697. Mol
---
[698, 699]

698. Sci
---
[699, 700]

699. Vet
---
[700, 701]

700. Bio
---
[701, 702]

701. Ana
---


[993, 998]

993. Int
---
[994, 999]

994. Bio
---
[995, 1000]

995. Dev
---
[996, 1001]

996. Inv
---
[997, 1002]

997. J E
---
[998, 1003]

998. Nat
---
[999, 1004]

999. PLo
---
[1000, 1005]

1000. Fr
---
[1001, 1006]

1001. Sc
---
[1002, 1007]

1002. Cu
---
[1003, 1008]

1003. Am
---
[1004, 1009]

1004. Pr
---
[1005, 1010]

1005. Ma
---
[1006, 1011]

1006. PL
---
[1007, 1012]

1007. Sc
---
[1008, 1013]

1008. J 
---
[1009, 1014]

1009. Ti
---
[1010, 1015]

1010. De
---
[1011, 1016]

1011. De
---
[1012, 1017]

1012. Ne
---
[1013, 1018]

1013. Sc
---
[1014, 1019]

1014. Pa
---
[1015, 1020]

1015. Im
---
[1016, 1021]

1016. He
---
[1017, 1022]

1017. Xe
---
[1018, 1023]

1018. Ep
---
[1019, 1024]

1019. J 
---
[1020, 1025]

1020. J 
---
[1021, 1026]

1021. Ce
---
[1022, 1027]

1022. Bi
---
[1023, 1028]

1023. J 
---
[1024, 1029]

1024. Sc
---
[1025, 1030]

1025. Re
---
[1026, 1031]

1026. J 
---
[1027, 1032]

1027. Ce
---
[1028, 1033]

1028. Wo
---
[1029, 1034]

1029. Pa
---
[1030, 103

[1301, 1306]

1301. Am
---
[1302, 1307]

1302. J 
---
[1303, 1308]

1303. La
---
[1304, 1309]

1304. Am
---
[1305, 1310]

1305. To
---
[1306, 1311]

1306. J 
---
[1307, 1312]

1307. Pr
---
[1308, 1313]

1308. Me
---
[1309, 1314]

1309. Em
---
[1310, 1315]

1310. Ge
---
[1311, 1316]

1311. Am
---
[1312, 1317]

1312. Am
---
[1313, 1318]

1313. J 
---
[1314, 1319]

1314. Na
---
[1315, 1320]

1315. Eu
---
[1316, 1321]

1316. Pa
---
[1317, 1322]

1317. Ne
---
[1318, 1323]

1318. In
---
[1319, 1324]

1319. Na
---
[1320, 1325]

1320. Br
---
[1321, 1326]

1321. Cu
---
[1322, 1327]

1322. Ne
---
[1323, 1328]

1323. Ne
---
[1324, 1329]

1324. Ph
---
[1325, 1330]

1325. Ne
---
[1326, 1331]

1326. Ph
---
[1327, 1332]

1327. Ex
---
[1328, 1333]

1328. On
---
[1329, 1334]

1329. Ne
---
[1330, 1335]

1330. Sc
---
[1331, 1336]

1331. Re
---
[1332, 1337]

1332. Am
---
[1333, 1338]

1333. Co
---
[1334, 1339]

1334. Cl
---
[1335, 1340]

1335. J 
---
[1336, 1341]

1336. J 
---
[1337, 1342]

1337. PL
---
[

...

[3753, 3787]

3753. Ps
---
[3754, 3789]

3754. Cl
---
[3755, 3790]

3755. Am
---
[3756, 3791]

3756. Pr
---
[3757, 3792]

3757. To
---
[3758, 3793]

3758. Hu
---
[3759, 3794]

3759. Ne
---
[3760, 3795]

3760. Ps
---
[3761, 3796]

3761. Bi
---
[3762, 3797]

3762. Im
---
[3763, 3798]

3763. To
---
[3764, 3799]

3764. To
---
[3765, 3800]

3765. J 
---
[3766, 3801]

3766. J 
---
[3767, 3802]

3767. J 
---
[3768, 3803]

3768. Vi
---
[3769, 3804]

3769. Vi
---
[3770, 3805]

3770. In
---
[3771, 3806]

3771. Bi
---
[3772, 3807]

3772. Ti
---
[3773, 3808]

3773. La
---
[3774, 3809]

3774. Me
---
[3775, 3810]

3775. La
---
[3776, 3811]

3776. Sy
---
[3777, 3812]

3777. J 
---
[3778, 3813]

3778. Ex
---
[3779, 3814]

3779. Bi
---
[3780, 3815]

3780. Na
---
[3781, 3816]

3781. Ac
---
[3782, 3817]

3782. La
---
[3783, 3818]

3783. La
---
[3784, 3819]

3784. Am
---
[3785, 3820]

3785. Pe
---
[3786, 3821]

3786. J 
---
[3787, 3822]

3787. J 
---
[3788, 3823]

3788. Tr
---
[3789, 3824]

3789. J 
---
[

[4058, 4101]

4058. Ex
---
[4059, 4102]

4059. Me
---
[4060, 4103]

4060. Me
---
[4061, 4104]

4061. Eu
---
[4062, 4105]

4062. An
---
[4063, 4106]

4063. AI
---
[4064, 4107]

4064. Be
---
[4065, 4108]

4065. Ac
---
[4066, 4109]

4066. J 
---
[4067, 4110]

4067. J 
---
[4068, 4111]

4068. Na
---
[4069, 4112]

4069. Ec
---
[4070, 4113]

4070. Br
---
[4071, 4114]

4071. Ne
---
[4072, 4115]

4072. Vi
---
[4073, 4116]

4073. Bi
---
[4074, 4117]

4074. J 
---
[4075, 4118]

4075. Ne
---
[4076, 4119]

4076. La
---
[4077, 4120]

4077. In
---
[4078, 4121]

4078. An
---
[4079, 4122]

4079. An
---
[4080, 4123]

4080. Bi
---
[4081, 4124]

4081. Eu
---
[4082, 4125]

4082. J 
---
[4083, 4126]

4083. J 
---
[4084, 4127]

4084. J 
---
[4085, 4128]

4085. Ph
---
[4086, 4129]

4086. J 
---
[4087, 4130]

4087. Pr
---
[4088, 4131]

4088. Mo
---
[4089, 4132]

4089. Bi
---
[4090, 4133]

4090. Ne
---
[4091, 4134]

4091. Vi
---
[4092, 4135]

4092. Vi
---
[4093, 4136]

4093. Ph
---
[4094, 4137]

4094. Am
---
[

[4363, 4413]

4363. Bi
---
[4364, 4414]

4364. Ve
---
[4365, 4415]

4365. In
---
[4366, 4416]

4366. La
---
[4367, 4417]

4367. Ni
---
[4368, 4418]

4368. Ph
---
[4369, 4419]

4369. Im
---
[4370, 4420]

4370. J 
---
[4371, 4421]

4371. Ex
---
[4372, 4422]

4372. Am
---
[4373, 4423]

4373. La
---
[4374, 4424]

4374. Bi
---
[4375, 4425]

4375. J 
---
[4376, 4426]

4376. Ar
---
[4377, 4427]

4377. Ex
---
[4378, 4428]

4378. Ar
---
[4379, 4429]

4379. An
---
[4380, 4430]

4380. Sc
---
[4381, 4431]

4381. Fo
---
[4382, 4432]

4382. Ar
---
[4383, 4433]

4383. Ar
---
[4384, 4435]

4384. Co
---
[4385, 4436]

4385. Ac
---
[4386, 4437]

4386. J 
---
[4387, 4438]

4387. Ar
---
[4388, 4439]

4388. Fo
---
[4389, 4440]

4389. La
---
[4390, 4441]

4390. Fo
---
[4391, 4442]

4391. Ad
---
[4392, 4443]

4392. Ch
---
[4393, 4444]

4393. Ex
---
[4394, 4445]

4394. Bi
---
[4395, 4446]

4395. Ne
---
[4396, 4447]

4396. Ri
---
[4397, 4448]

4397. La
---
[4398, 4449]

4398. J 
---
[4399, 4450]

4399. Hi
---
[

[4669, 4721]

4669. La
---
[4670, 4722]

4670. La
---
[4671, 4723]

4671. La
---
[4672, 4724]

4672. Cl
---
[4673, 4725]

4673. Ar
---
[4674, 4726]

4674. La
---
[4675, 4727]

4675. Ce
---
[4676, 4728]

4676. Ve
---
[4677, 4729]

4677. J 
---
[4678, 4730]

4678. J 
---
[4679, 4731]

4679. De
---
[4680, 4732]

4680. J 
---
[4681, 4733]

4681. Am
---
[4682, 4734]

4682. Bi
---
[4683, 4735]

4683. FE
---
[4684, 4736]

4684. Am
---
[4685, 4737]

4685. Am
---
[4686, 4738]

4686. Am
---
[4687, 4739]

4687. Am
---
[4688, 4740]

4688. Am
---
[4689, 4741]

4689. Ad
---
[4690, 4742]

4690. Pr
---
[4691, 4743]

4691. Fo
---
[4692, 4744]

4692. Ps
---
[4693, 4745]

4693. Fo
---
[4694, 4746]

4694. Ne
---
[4695, 4747]

4695. J 
---
[4696, 4748]

4696. Im
---
[4697, 4749]

4697. J 
---
[4698, 4750]

4698. Cl
---
[4699, 4751]

4699. An
---
[4700, 4752]

4700. Co
---
[4701, 4753]

4701. Co
---
[4702, 4754]

4702. An
---
[4703, 4755]

4703. J 
---
[4704, 4756]

4704. J 
---
[4705, 4757]

4705. Bi
---
[

[4975, 5033]

4975. Cy
---
[4976, 5034]

4976. Ar
---
[4977, 5035]

4977. J 
---
[4978, 5036]

4978. J 
---
[4979, 5037]

4979. La
---
[4980, 5038]

4980. Bi
---
[4981, 5039]

4981. Ac
---
[4982, 5040]

4982. An
---
[4983, 5041]

4983. Pr
---
[4984, 5042]

4984. AM
---
[4985, 5043]

4985. AM
---
[4986, 5044]

4986. J 
---
[4987, 5045]

4987. Am
---
[4988, 5046]

4988. En
---
[4989, 5047]

4989. La
---
[4990, 5048]

4990. Cy
---
[4991, 5049]

4991. La
---
[4992, 5050]

4992. J 
---
[4993, 5051]

4993. Am
---
[4994, 5052]

4994. J 
---
[4995, 5053]

4995. Bi
---
[4996, 5054]

4996. Bi
---
[4997, 5055]

4997. La
---
[4998, 5056]

4998. Cy
---
[4999, 5057]

4999. Ac
---
[5000, 5058]

5000. Sc
---
[5001, 5059]

5001. Re
---
[5002, 5060]

5002. Cy
---
[5003, 5061]

5003. En
---
[5004, 5062]

5004. Pr
---
[5005, 5063]

5005. Bi
---
[5006, 5064]

5006. Sc
---
[5007, 5065]

5007. Am
---
[5008, 5066]

5008. Bi
---
[5009, 5067]

5009. J 
---
[5010, 5068]

5010. J 
---
[5011, 5069]

5011. Am
---
[

In [68]:
i>=len(blockED)-1

True

In [57]:
print(len(blockED))
print(blockED[5082-1])
print(i)
blockED[i-1]

5082
9635742
5081


9634653

In [59]:
i=len(blockED)
i

5082

In [71]:
fname_tag="mpd02"
fext=".txt"

fileList = os.listdir(fname_tag)
fileList2 = [f for f in fileList if os.path.isfile(os.path.join(fname_tag, f)) and fext in f]
print(fileList2)
print(fileList2[:,:])
print(len(fileList2))

['3644.txt', '1053.txt', '1735.txt', '4895.txt', '3122.txt', '4881.txt', '2228.txt', '3136.txt', '4659.txt', '1721.txt', '1047.txt', '3650.txt', '3888.txt', '4117.txt', '3678.txt', '2566.txt', '2200.txt', '4671.txt', '1709.txt', '289.txt', '2214.txt', '4665.txt', '4103.txt', '2572.txt', '504.txt', '1090.txt', '2599.txt', '3687.txt', '4856.txt', '262.txt', '276.txt', '4842.txt', '3693.txt', '1084.txt', '510.txt', '3863.txt', '538.txt', '1912.txt', '1906.txt', '3877.txt', '3320.txt', '1537.txt', '1251.txt', '2758.txt', '3446.txt', '2980.txt', '4329.txt', '5023.txt', '3452.txt', '2994.txt', '909.txt', '1245.txt', '1523.txt', '3334.txt', '4473.txt', '2002.txt', '2764.txt', '4315.txt', '921.txt', '935.txt', '1279.txt', '2770.txt', '4301.txt', '4467.txt', '3308.txt', '2016.txt', '3485.txt', '2943.txt', '1292.txt', '706.txt', '712.txt', '1286.txt', '3491.txt', '2957.txt', '4498.txt', '3254.txt', '1443.txt', '29.txt', '1325.txt', '869.txt', '3532.txt', '4249.txt', '3526.txt', '2638.txt', '1331

TypeError: list indices must be integers or slices, not tuple

In [72]:
# print(fileList2)
fNum=[int(f[:-len(fext)]) for f in fileList2]
print(fNum)
fNum.sort()
print('---')
print(fNum)



[3644, 1053, 1735, 4895, 3122, 4881, 2228, 3136, 4659, 1721, 1047, 3650, 3888, 4117, 3678, 2566, 2200, 4671, 1709, 289, 2214, 4665, 4103, 2572, 504, 1090, 2599, 3687, 4856, 262, 276, 4842, 3693, 1084, 510, 3863, 538, 1912, 1906, 3877, 3320, 1537, 1251, 2758, 3446, 2980, 4329, 5023, 3452, 2994, 909, 1245, 1523, 3334, 4473, 2002, 2764, 4315, 921, 935, 1279, 2770, 4301, 4467, 3308, 2016, 3485, 2943, 1292, 706, 712, 1286, 3491, 2957, 4498, 3254, 1443, 29, 1325, 869, 3532, 4249, 3526, 2638, 1331, 1457, 3240, 2176, 3268, 4507, 4261, 2610, 15, 1319, 855, 699, 841, 4275, 2604, 2162, 4513, 1480, 114, 3297, 2189, 2837, 672, 666, 2823, 3283, 100, 1494, 128, 896, 882, 3730, 1127, 1641, 1899, 4739, 3056, 2348, 3042, 1655, 1133, 3724, 2412, 4063, 4705, 2374, 1669, 4711, 2360, 2406, 3718, 4077, 470, 4922, 3095, 316, 1682, 1696, 302, 4936, 3081, 4088, 464, 3917, 1866, 1872, 458, 3903, 1873, 3902, 459, 3916, 1867, 3080, 4937, 303, 1697, 465, 4089, 471, 1683, 317, 3094, 4923, 2361, 4710, 1668, 4076, 371

In [91]:
k=271
fin_name=os.path.join(fname_tag,str(k+1)+fext)
with open(fin_name, "r", encoding="utf-8") as fid:
    strReadD=fid.read()

print(strReadD)

blockPos=[]
patternRep="\n\n"
res_iter =re.finditer(patternRep, strReadD)
for res in res_iter:
    Mstart, Mend = res.span()
    blockPos.append([Mstart,Mend])

blockPos



272. J Neural Transm (Vienna). 2020 Oct;127(10):1343-1358. doi:
10.1007/s00702-020-02247-2. Epub 2020 Aug 29.

The MPTP-lesioned marmoset model of Parkinson's disease: proposed efficacy
thresholds that may potentially predict successful clinical trial results.

Beaudry F(1), Huot P(2)(3)(4).

Author information: 
(1)Groupe de Recherche en Pharmacologie Animal du Québec, Département de
Biomédecine Vétérinaire, Faculté de Médecine Vétérinaire, Université de Montréal,
Saint-Hyacinthe, QC, Canada.
(2)Neurodegenerative Disease Group, Montreal Neurological Institute, 3801
University St, Montreal, QC, H3A 2B4, Canada. philippe.huot@mcgill.ca.
(3)Department of Neurology and Neurosurgery, McGill University, Montreal, QC,
Canada. philippe.huot@mcgill.ca.
(4)Movement Disorder Clinic, Division of Neurology, Department of Neurosciences, 
McGill University Health Centre, Montreal, QC, Canada. philippe.huot@mcgill.ca.

The 1-methyl-4-phenyl-1,2,3,6-tetrahydropyridine (MPTP)-lesioned common marmoset

[[0, 2],
 [111, 113],
 [262, 264],
 [294, 296],
 [918, 920],
 [3229, 3231],
 [3301, 3303]]

In [92]:
for i in range(len(blockPos)-1):
#     print(blockPos[i][0],blockPos[i][1])
    print(blockPos[i][1],blockPos[i+1][0])
    print(strReadD[blockPos[i][1]:blockPos[i+1][0]])
    print('---')

2 111
272. J Neural Transm (Vienna). 2020 Oct;127(10):1343-1358. doi:
10.1007/s00702-020-02247-2. Epub 2020 Aug 29.
---
113 262
The MPTP-lesioned marmoset model of Parkinson's disease: proposed efficacy
thresholds that may potentially predict successful clinical trial results.
---
264 294
Beaudry F(1), Huot P(2)(3)(4).
---
296 918
Author information: 
(1)Groupe de Recherche en Pharmacologie Animal du Québec, Département de
Biomédecine Vétérinaire, Faculté de Médecine Vétérinaire, Université de Montréal,
Saint-Hyacinthe, QC, Canada.
(2)Neurodegenerative Disease Group, Montreal Neurological Institute, 3801
University St, Montreal, QC, H3A 2B4, Canada. philippe.huot@mcgill.ca.
(3)Department of Neurology and Neurosurgery, McGill University, Montreal, QC,
Canada. philippe.huot@mcgill.ca.
(4)Movement Disorder Clinic, Division of Neurology, Department of Neurosciences, 
McGill University Health Centre, Montreal, QC, Canada. philippe.huot@mcgill.ca.
---
920 3229
The 1-methyl-4-phenyl-1,2,3,6-t

In [96]:
i=4
absStr=strReadD[blockPos[i][1]:blockPos[i+1][0]]
absStrRep=absStr.replace('\n', ' ')
absStrRep=absStrRep.replace('  ', ' ')
print(absStrRep)


The 1-methyl-4-phenyl-1,2,3,6-tetrahydropyridine (MPTP)-lesioned common marmoset has been used extensively to model Parkinson's disease, L-3,4-dihydroxyphenylalanine (L-DOPA)-induced dyskinesia and, more recently, dopaminergic psychosis. Whereas several experimental drugs have been tested in this primate, many of which subsequently underwent clinical trials, efficacy thresholds in the marmoset that would predict efficacy in the clinic are lacking. Here, we aimed to determine such efficacy end points that would be indicative of likely efficacy in clinical studies. To do so, we used the evidence-based medicine reviews published by the International Parkinson and Movement Disorder Society (IPMDS) to select drugs that were rated as clinically efficacious, likely efficacious or not efficacious for the treatment of parkinsonism, dyskinesia and psychosis. We then reviewed the literature in the MPTP-lesioned marmoset and identified articles reporting the effects of drugs that were included in 